In [3]:
def linear_anneal(t, T, start, final, percentage):
    ''' Linear annealing scheduler
    t: current timestep
    T: total timesteps
    start: initial value
    final: value after percentage*T steps
    percentage: percentage of T after which annealing finishes
    '''
    final_from_T = int(percentage * T)
    if t > final_from_T:
        return final
    else:
        return final + (start - final) * (final_from_T - t) / final_from_T

In [24]:
linear_anneal(17, 25, 1.0, 0.01, 0.8)

0.45999999999999996

In [16]:
import torch.nn as nn


def build_nn():
    input_size = 72
    output = 8
    layer_sizes = [input_size] + [128, 128, 128] + [output]

    assert len(layer_sizes) > 1
    layers = []
    for index in range(len(layer_sizes) - 1):
        linear = nn.Linear(layer_sizes[index], layer_sizes[index + 1])
        act = nn.ReLU() if index < len(layer_sizes) - 2 else nn.Identity()
        layers += (linear, act)

    return nn.Sequential(*layers)

In [24]:
import random

In [31]:
import pandas as pd
import numpy as np

ddf2 = pd.DataFrame(np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]]),
                    columns=['a', 'b', 'c'])
df2.to_csv()

NameError: name 'df' is not defined